# Libraries

In [1]:
#Libraries
import gymnasium as gym
from tetris_gymnasium.envs.tetris import Tetris
from tetris_gymnasium.envs.tetris import TetrisState
from tetris_gymnasium.wrappers.observation import FeatureVectorObservation

import sys
import cv2
import time
import numpy as np

import os

import neat
import random
import pickle

# Load the Neural Network from a Pickle File

In [2]:
# Load the neural network from the pkl file

with open('my_winner_delh_only.pkl', 'rb') as f:
     x_net_delh_only= pickle.load(f)
with open('my_winner.pkl', 'rb') as f:
     x_net_del_h_andholes = pickle.load(f)
print(x_net_del_h_andholes)
print(x_net_delh_only)

# Visualize Random Agent --> Press 'a' to start the clip

In [11]:
if __name__ == "__main__":
    # Create an instance of Tetris
    env = gym.make("tetris_gymnasium/Tetris", render_mode="human")
    env.reset(seed=6252590) ########CHANGE SEED
    reward_hist = 0
    del_h_hist = 0
    del_holes_hist = 0 
    start_clip =0
    # Main game loop
    terminated = False
    while not terminated:
        env.render()
        action = None
        while action is None:
            key = cv2.waitKey(1)
            
            if start_clip==0:
                if key == ord("a"):
                    action = env.unwrapped.actions.move_left
                    start_clip=1
            else:        
                env1 = FeatureVectorObservation(env) #create an instance of the class

                height_i = env1.calc_height(env.board) #call a function from the class with the board as an input
                height_i = (height_i[4:14])-4 #crop out the padding on either side (the bedrock) and normalize by the 4 layers of bedrock underneath
                prev_height = height_i #calculate the height before the move

                ### Code to get the holes before the move 
                holes_i = env1.calc_holes(env.board)
                prev_holes = holes_i #calculate the holes before the move

                ### FEATURIZATION
                
                ### Get the current tetromino in a list form and in the position relative to the other blocks
                tetromino = env.active_tetromino
                slices = env.get_tetromino_slices(tetromino, env.x, env.y)
                #Create pseudo_board of just zeros
                only_tetromino = np.zeros((24,18))
                #add the tetromino in its correct place
                only_tetromino[slices] += tetromino.matrix
                #Convert to 1s and 0s
                only_tetromino = np.where(only_tetromino > 0, 1, 0)
                #Sum in each column
                only_tetromino = np.sum(only_tetromino,axis=0)
                #Crop out the empty bedrock
                only_tetromino=only_tetromino[4:14]
                #print(only_tetromino)

                
                #compile the two features together
                i_feat = np.hstack((only_tetromino,height_i)) #this is a one dimensional list of 20 integers

                ### FEED THE FEATURES INTO THE NEURAL NETWORK
                
                #Iterate to the next step
                action = np.argmax(x_net_del_h_andholes.activate(i_feat)) ### CHANGE WITH NETWORK PLAYING WITH
                if action ==2:
                    action = 5

            if (
                cv2.getWindowProperty(env.unwrapped.window_name, cv2.WND_PROP_VISIBLE)
                == 0
                ):
                sys.exit()
        
        time.sleep(.1) #add a delay in between each move for better visualization
        
        # Perform the action
        observation, reward, terminated, truncated, info = env.step(action)


    # Game over
    print("Game Over!")

Game Over!
